<a id='up'></a>
# Проект PostgreSQL. Оконные функции и когортный анализ

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Описание-проекта" data-toc-modified-id="Описание-проекта-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Описание проекта</a></span></li><li><span><a href="#Загрузка-библиотек-и-подключение-к-серверу" data-toc-modified-id="Загрузка-библиотек-и-подключение-к-серверу-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Загрузка библиотек и подключение к серверу</a></span></li><li><span><a href="#Задачи-на-оконные-функции" data-toc-modified-id="Задачи-на-оконные-функции-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Задачи на оконные функции</a></span></li><li><span><a href="#Когортный-анализ" data-toc-modified-id="Когортный-анализ-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Когортный анализ</a></span></li><li><span><a href="#Общий-вывод" data-toc-modified-id="Общий-вывод-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Общий вывод</a></span></li></ul></div>

## Описание проекта

В данном проекте я буду выполнять задачи на отработку навыков владения PostgreSQL, в частности задачи на применение оконных функций и проведение когортного анализа. В проекте я буду проводить анализ данных интернет-магазина и рассчитывать такие показатели как: LTV, ROI, ARPPU, коэффициенты удержания и оттока пользователей магазина средствами PostgreSQL.

**План проекта:**
- Загрузка библиотек и подключение к серверу
- Задачи на оконные функции
- Когортный анализ
- Общий вывод

**Описание данных:**
1. Таблица users:

Cодержит данные о пользователях.

- user_id - Идентификатор пользователя, первичный ключ таблицы
- first_name - Имя пользователя
- last_name - Фамилия пользователя
- email	- Электронный адрес пользователя
- created_at - Дата создания аккаунта пользователя
2. Таблица events:

Содержит данные о событиях.

- event_id - Идентификатор события, первичный ключ таблицы
- event_time - Дата и время события
- event_name - Название события
- user_id - Идентификатор пользователя, внешний ключ, отсылающий к таблице users
- platform - Тип устройства пользователя
- referrer - Источник события пользователя

Типов событий в поле event_name два: 
- view_item — просмотр товара;
- view_user_profile — просмотр сохранённых товаров другого пользователя.

Значение в поле referrer показывает, какой источник просматривал пользователь перед тем, как совершилось событие. Источники могут быть внешними:
- google_search — поисковая система Google;
- promo_email_click — рассылка магазина.

В поле referrer могут быть указаны и источники внутри сайта:
- item_page — страница товара;
- user_wishlist — страница сохранённых товаров другого пользователя;
- shopping_cart — корзина;
- home — главная страница магазина.

3. Таблица event_x_event_parameter:

Содержит детальную информацию о событиях.

- event_id - Идентификатор события, внешний ключ, отсылающий к таблице events
- parameter_name - Параметр детализации
- parameter_value - Значение параметра

Если в поле parameter_name указан item_id, то в parameter_value будет храниться идентификатор товара, который просматривал пользователь.
Если в поле parameter_name указан viewed_user_id, то в parameter_value будет храниться идентификатор пользователя, чью страницу с сохранёнными товарами смотрел другой пользователь.

4. Таблица orders:

Содержит детальные данные о заказах пользователей.

- order_id - Идентификатор заказа, первичный ключ таблицы
- user_id - Идентификатор пользователя, оформившего заказ
- total_amt	- Общая сумма заказа
- items_cnt	- Количество товаров в заказе
- created_at - Дата, когда пользователь оформил заказ, но ещё не оплатил его
- paid_at - Дата оплаты заказа

5. Таблица items:

Содержит данные о товарах.

- item_id - Идентификатор товара, первичный ключ таблицы
- category - Категория товара
- item_name - Название товара
- price - Стоимость

6. Таблица order_x_item:

Содержит данные о связи заказа с товарами.

- order_id - Идентификатор заказа, внешний ключ, отсылающий к таблице orders
- item_id - Идентификатор товара, внешний ключ, отсылающий к таблице items

7. Таблица costs:

Содержит данные о стоимости привлечения пользователей.

- created_at - Дата
- costs - Стоимость привлечения

## Загрузка библиотек и подключение к серверу

Импорт библиотек:

In [1]:
import pandas as pd
from sqlalchemy import create_engine
from dotenv import dotenv_values

Подключение к серверу:

In [2]:
path = 'D:\\yandeks\\projects\\13_postgresql\\pass.env'
config = dotenv_values(path)

In [3]:
db_config = {
    'user': config['user'], # имя пользователя
    'pwd': config['pwd'], # пароль
    'host': config['host'],
    'port': config['port'], # порт подключения
    'db': config['db'] # название базы данных
}
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)
engine = create_engine(connection_string)

Функция для вывода результатов запроса:

In [4]:
def query_sql(query):
    display(pd.read_sql_query(query, con = engine))

## Задачи на оконные функции

1. Написать запрос, который проранжирует расходы на привлечение пользователей за каждый день по убыванию. Ранги не должны повторяться.
Выгрузить три поля: 
- дата, которую нужно привести к типу date;
- расходы на привлечение;
- ранг записи.

In [5]:
query0 = '''
SELECT CAST(created_at AS date),
        costs,
        ROW_NUMBER() OVER(ORDER BY costs DESC)
FROM tools_shop.costs;
'''
query_sql(query0)

,created_at,costs,row_number
0,2019-12-16,3247.58,1
1,2021-02-26,3121.92,2
2,2021-04-17,3113.96,3
3,2021-03-11,3087.05,4
4,2021-05-06,2934.81,5
...,...,...,...
1814,2016-07-06,1.99,1815
1815,2016-07-08,1.76,1816
1816,2016-10-25,1.59,1817
1817,2016-05-27,0.83,1818


2. Изменить предыдущий запрос: записям с одинаковыми значениями расходов назначить одинаковый ранг. Ранги не должны прерываться.

In [6]:
query0 = '''
SELECT CAST(created_at AS date),
        costs,
        DENSE_RANK() OVER(ORDER BY costs DESC)
FROM tools_shop.costs;
'''
query_sql(query0)

,created_at,costs,dense_rank
0,2019-12-16,3247.58,1
1,2021-02-26,3121.92,2
2,2021-04-17,3113.96,3
3,2021-03-11,3087.05,4
4,2021-05-06,2934.81,5
...,...,...,...
1814,2016-07-06,1.99,1803
1815,2016-07-08,1.76,1804
1816,2016-10-25,1.59,1805
1817,2016-05-27,0.83,1806


3. Используя оконную функцию, вывести список уникальных user_id пользователей, которые оформили три заказа и более.

In [7]:
query0 = '''
WITH us AS (SELECT user_id,
        COUNT(*) OVER(PARTITION BY user_id) AS cou
    FROM tools_shop.orders)
SELECT DISTINCT user_id
FROM us
WHERE cou>=3;
'''
query_sql(query0)

,user_id
0,2950
1,3039
2,3763
3,7651
4,8468
...,...
77,247299
78,252685
79,253319
80,259210


4. Используя оконную функцию, вывести количество заказов, в которых было четыре товара и более.

In [8]:
query0 = '''
SELECT COUNT(*)
FROM tools_shop.orders
WHERE items_cnt>=4;
'''
query_sql(query0)

,count
0,4747


5. Рассчитать количество зарегистрированных пользователей по месяцам с накоплением.
Выгрузить два поля:
- месяц регистрации, приведённый к типу date;
- общее количество зарегистрированных пользователей на текущий месяц.

In [9]:
query0 = '''
WITH dr AS (SELECT CAST(DATE_TRUNC('month', created_at) AS date) AS month,
        COUNT(DISTINCT user_id) AS cou
FROM tools_shop.users
GROUP BY month)
SELECT month,
    SUM(cou) OVER(ORDER BY month)
FROM dr;
'''
query_sql(query0)

,month,sum
0,2016-02-01,9.0
1,2016-03-01,162.0
2,2016-04-01,450.0
3,2016-05-01,826.0
4,2016-06-01,1307.0
...,...,...
60,2021-02-01,108502.0
61,2021-03-01,111420.0
62,2021-04-01,114203.0
63,2021-05-01,117082.0


6. Рассчитать сумму трат на привлечение пользователей с накоплением по месяцам с 2017 по 2018 год включительно.
Выгрузить два поля:
- месяц, приведённый к типу date;
- сумма трат на текущий месяц с накоплением.

In [10]:
query0 = '''
WITH sd AS (SELECT CAST(DATE_TRUNC('month', created_at) AS date) AS month,
        SUM(costs) as costs
FROM tools_shop.costs
WHERE CAST(DATE_TRUNC('month', created_at) AS date) BETWEEN '2017-01-01' AND '2018-12-31'
GROUP BY month)
SELECT month,
        SUM(costs) OVER(ORDER BY month)
FROM sd;
'''
query_sql(query0)

,month,sum
0,2017-01-01,8054.75
1,2017-02-01,16472.46
2,2017-03-01,26315.39
3,2017-04-01,40632.33
4,2017-05-01,53851.92
5,2017-06-01,69186.32
6,2017-07-01,84158.57
7,2017-08-01,99067.96
8,2017-09-01,113845.30
9,2017-10-01,129929.17


7. Посчитать события с названием view_item по месяцам с накоплением. Найти количество событий только для тех пользователей, которые совершили хотя бы одну покупку.
Выгрузить поля: 
- месяц события, приведённый к типу date;
- количество событий за текущий месяц;
- количество событий за текущий месяц с накоплением.

In [11]:
query0 = '''
SELECT DATE_TRUNC('month', event_time)::date AS month,
        COUNT(*),
        SUM(COUNT(*)) OVER(ORDER BY DATE_TRUNC('month', event_time)::date)
FROM tools_shop.events AS e
WHERE event_name = 'view_item' AND user_id IN (SELECT DISTINCT user_id
                                                FROM tools_shop.orders AS o)
GROUP BY month;
'''
query_sql(query0)

,month,count,sum
0,2016-03-01,10,10.0
1,2016-04-01,23,33.0
2,2016-05-01,84,117.0
3,2016-06-01,111,228.0
4,2016-07-01,190,418.0
...,...,...,...
59,2021-02-01,1966,75857.0
60,2021-03-01,2059,77916.0
61,2021-04-01,1824,79740.0
62,2021-05-01,1701,81441.0


8. Используя конструкцию WINDOW, рассчитать суммарную стоимость заказов и количество заказов с накоплением от месяца к месяцу.
Выгрузить поля:
- идентификатор заказа;
- месяц оплаты заказа, приведённый к типу date;
- сумма заказа;
- количество заказов с накоплением;
- суммарная стоимость заказов с накоплением.

In [12]:
query0 = '''
SELECT order_id,
        DATE_TRUNC('month', paid_at)::date as date,
        total_amt,
        COUNT(*) OVER my AS count,
        SUM(total_amt) OVER me
FROM tools_shop.orders
WINDOW my AS (ORDER BY DATE_TRUNC('month', paid_at)::date),
        me AS (ORDER BY DATE_TRUNC('month', paid_at)::date);
'''
query_sql(query0)

,order_id,date,total_amt,count,sum
0,324983,2016-03-01,48.40,3,500.40
1,280567,2016-03-01,387.50,3,500.40
2,278543,2016-03-01,64.50,3,500.40
3,391902,2016-04-01,118.80,10,937.37
4,30978,2016-04-01,26.25,10,937.37
...,...,...,...,...,...
18966,213243,2021-06-01,53.40,18971,4826455.67
18967,50920,2021-06-01,39.60,18971,4826455.67
18968,18492,2021-06-01,420.00,18971,4826455.67
18969,223455,2021-06-01,8.82,18971,4826455.67


9. Написать запрос, который выведет сумму трат на привлечение пользователей по месяцам, а также разницу в тратах между текущим и предыдущим месяцами. Разница должна показывать, на сколько траты текущего месяца отличаются от предыдущего. В случае, если данных по предыдущему месяцу нет, указать ноль.
Выгрузить поля:
- месяц, приведённый к типу date;
- траты на привлечение пользователей в текущем месяце;
- разница в тратах между текущим и предыдущим месяцами.

In [13]:
query0 = '''
SELECT DATE_TRUNC('month', created_at)::date as date,
        SUM(costs),
        SUM(costs) - LAG(SUM(costs), 1, SUM(costs)) OVER(ORDER BY DATE_TRUNC('month', created_at)::date) as difference
FROM tools_shop.costs
GROUP BY DATE_TRUNC('month', created_at)::date;
'''
query_sql(query0)

,date,sum,difference
0,2016-03-01,130.59,0.00
1,2016-04-01,157.17,26.58
2,2016-05-01,1006.73,849.56
3,2016-06-01,1979.74,973.01
4,2016-07-01,4021.09,2041.35
...,...,...,...
59,2021-02-01,37628.07,-1375.53
60,2021-03-01,38275.16,647.09
61,2021-04-01,36832.95,-1442.21
62,2021-05-01,36974.49,141.54


10. Написать запрос, который выведет сумму выручки по годам и разницу выручки между текущим и следующим годом. Разница должна показывать, на сколько выручка следующего года отличается от текущего. В случае, если данных по следующему году нет, указать ноль.
Выгрузить поля:
- год, приведённый к типу date;
- выручка за текущий год;
- разница в выручке между текущим и следующим годом.

In [14]:
query0 = '''
SELECT DATE_TRUNC('year', paid_at)::date as date,
        SUM(total_amt),
        LEAD(SUM(total_amt), 1, SUM(total_amt)) OVER(ORDER BY DATE_TRUNC('year', paid_at)::date) - SUM(total_amt) as difference
FROM tools_shop.orders
GROUP BY DATE_TRUNC('year', paid_at)::date;
'''
query_sql(query0)

,date,sum,difference
0,2016-01-01,136017.66,429736.05
1,2017-01-01,565753.71,336605.77
2,2018-01-01,902359.48,258161.33
3,2019-01-01,1160520.81,262563.06
4,2020-01-01,1423083.87,-784363.73
5,2021-01-01,638720.14,0.00


## Когортный анализ

В этом разделе определим профили пользователей, рассчитаем конверсию, ARPPU, удержание, отток, LTV и ROI.

1. Написать запрос для определения профиля пользователя. Для этого на экране отобразить три поля: 
- идентификатор пользователя;
- номер месяца оплаты товара;
- номер месяца, когда пользователь впервые оплатил товар.

In [15]:
query0 = '''
SELECT user_id,
        EXTRACT(month FROM paid_at),
        MIN(EXTRACT(month FROM paid_at)) OVER(PARTITION BY user_id ORDER BY EXTRACT(month FROM paid_at))
FROM tools_shop.orders
WHERE paid_at BETWEEN '2020-01-01' AND '2021-01-01';
'''
query_sql(query0)

,user_id,extract,min
0,136148,1.0,1.0
1,137472,1.0,1.0
2,137733,1.0,1.0
3,138124,1.0,1.0
4,138291,1.0,1.0
...,...,...,...
5409,256621,12.0,12.0
5410,256818,12.0,12.0
5411,257150,12.0,12.0
5412,257162,12.0,12.0


2. Определить общий показатель конверсии в покупателя:

In [16]:
query = '''
SELECT ROUND(COUNT(DISTINCT o.user_id) *100.0 / COUNT(DISTINCT u.user_id), 1) as cr
FROM tools_shop.users u
LEFT JOIN tools_shop.orders o ON u.user_id = o.user_id;
'''
query_sql(query)

,cr
0,14.9


3. Рассчитать ARPPU в разрезе года оформления заказа. Поле с датой привести к типу данных date, а значение ARPPU округлить до двух знаков после запятой. Выводить в таком порядке: сначала поле с датой, затем поле с метрикой ARPPU.

In [17]:
query0 = '''
SELECT CAST (DATE_TRUNC('year', o.created_at) AS date) AS year,
    ROUND(SUM(o.total_amt)/COUNT(DISTINCT o.user_id), 2) as arppu
FROM tools_shop.orders AS o
GROUP BY year;
'''
query_sql(query)

,cr
0,14.9


4. Подсчитать Retention Rate. Отобразить три поля:
- когорту;
- месяц;
- Retention Rate, округлённый до четырёх знаков после запятой и умноженный на 100.

Отсортировать таблицу по когорте и по месяцу.

In [18]:
query0= '''
WITH profiles AS (SELECT user_id,
        EXTRACT(month FROM paid_at) as mon,
        MIN(EXTRACT(month FROM paid_at)) OVER(PARTITION BY user_id ORDER BY EXTRACT(month FROM paid_at)) AS coh
FROM tools_shop.orders
WHERE paid_at BETWEEN '2020-01-01' AND '2021-01-01'),

ret AS (SELECT coh,
        mon,
        COUNT (DISTINCT user_id) as us
FROM profiles
GROUP BY coh, mon)
SELECT coh,
        mon,
        ROUND(us::numeric/MAX(us::numeric) OVER(PARTITION BY coh), 4)*100 as rr
FROM ret;
'''
query_sql(query0)

,coh,mon,rr
0,1.0,1.0,100.00
1,1.0,2.0,2.48
2,1.0,3.0,0.74
3,1.0,4.0,0.74
4,1.0,5.0,1.24
5,1.0,6.0,0.74
6,2.0,2.0,100.00
7,2.0,3.0,1.96
8,2.0,4.0,1.68
9,2.0,5.0,1.68


5. Подсчет Churn Rate:

In [19]:
query0 = '''
WITH pro AS (SELECT user_id,
        MIN(DATE_TRUNC('month', event_time))::date as month
    FROM tools_shop.events
    WHERE user_id IN (SELECT DISTINCT user_id
                  FROM tools_shop.orders)
    GROUP BY user_id),

eve AS (SELECT month,
        DATE_TRUNC('month', event_time)::date AS event,
        COUNT (DISTINCT ev.user_id) as users
    FROM pro
    JOIN tools_shop.events AS ev ON pro.user_id=ev.user_id
    GROUP BY month, DATE_TRUNC('month', event_time)::date)
SELECT *,
        LAG(users) OVER(PARTITION BY month ORDER BY event),
        ROUND(1.0-(users::numeric/LAG(users) OVER(PARTITION BY month ORDER BY event)::numeric), 4) *100 as churn_rate
FROM eve;
'''
query_sql(query0)

,month,event,users,lag,churn_rate
0,2016-03-01,2016-03-01,4,NaN,NaN
1,2016-03-01,2016-07-01,2,4.0,50.00
2,2016-03-01,2016-08-01,1,2.0,50.00
3,2016-03-01,2016-09-01,2,1.0,-100.00
4,2016-04-01,2016-04-01,12,NaN,NaN
...,...,...,...,...,...
611,2021-04-01,2021-05-01,73,372.0,80.38
612,2021-04-01,2021-06-01,4,73.0,94.52
613,2021-05-01,2021-05-01,313,NaN,NaN
614,2021-05-01,2021-06-01,6,313.0,98.08


6. Подсчет LTV:

In [20]:
query0 = '''
WITH prof AS (SELECT user_id,
                     DATE_TRUNC('month', created_at)::date AS coh,
              COUNT(user_id) OVER(PARTITION BY DATE_TRUNC('month', created_at)::date) AS coh_size
              FROM tools_shop.users),
ord AS (SELECT EXTRACT(MONTH FROM AGE(DATE_TRUNC('month', o.created_at)::date, coh::date)) AS life,
               DATE_TRUNC('month', o.created_at)::date AS ord,
               coh,
               coh_size,
               total_amt
        FROM prof
        JOIN tools_shop.orders AS o ON prof.user_id=o.user_id),
ltv AS (SELECT life,
               coh,
               coh_size,
               SUM(total_amt) OVER(PARTITION BY coh ORDER BY life) / coh_size AS ltv
        FROM ord)
SELECT life,
       coh,
       ltv
FROM ltv
WHERE coh BETWEEN '2019-01-01' AND '2019-12-31'
GROUP BY life, coh, ltv
ORDER BY coh, life;
'''
query_sql(query0)

,life,coh,ltv
0,0.0,2019-01-01,2.383014
1,1.0,2019-01-01,11.821655
2,2.0,2019-01-01,19.622368
3,3.0,2019-01-01,25.247877
4,4.0,2019-01-01,31.643873
...,...,...,...
91,3.0,2019-12-01,26.198193
92,4.0,2019-12-01,32.198580
93,5.0,2019-12-01,39.006749
94,6.0,2019-12-01,46.621850


7. Подсчет ROI:

In [21]:
query0 = '''
SELECT dc.dt,
        total_amt*100.0 / costs AS roi
FROM
(SELECT DATE_TRUNC('month', created_at)::date AS dt,
       SUM(costs) AS costs
FROM tools_shop.costs
GROUP BY dt) AS dc
JOIN
(SELECT DATE_TRUNC('month', paid_at)::date AS dt,
       SUM(total_amt) AS total_amt
FROM tools_shop.orders
GROUP BY dt) AS ta ON dc.dt = ta.dt;
'''
query_sql(query0)

,dt,roi
0,2018-09-01,314.003867
1,2018-01-01,325.014409
2,2019-12-01,361.172492
3,2018-06-01,332.421390
4,2020-07-01,325.040600
...,...,...
59,2021-06-01,543.230295
60,2016-10-01,322.192174
61,2017-08-01,323.020325
62,2018-05-01,384.321590


## Общий вывод

- В проекте я отработал навыки владения PostgreSQL, выполнил задачи на применение оконных функций и провел когортный анализ пользователей интернет-магазина. Я определил профили пользователей, рассчитал такие показатели как: LTV, ROI, ARPPU, коэффициенты удержания, оттока пользователей магазина и конверсию в покупателя.

[В начало проекта](#up)